In [1]:
import os
from google.cloud import bigquery

os.environ.setdefault("GCLOUD_PROJECT", "enduring-rush-410922")
client = bigquery.Client()

/Users/mitch/envs/main/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [44]:
QUERY = """
WITH country_data AS (
  SELECT country_code, 
  short_name Name,
  region Region, 
  income_group `Income Group`
  FROM bigquery-public-data.world_bank_wdi.country_summary
)
SELECT data.country_code, Name,
CASE
    WHEN indicator_code = "NE.CON.TOTL.CD" THEN "Consumption"
    WHEN indicator_code = "NE.CON.TOTL.CN" THEN "Consumption_alt"
    --WHEN indicator_code = "NE.GDI.FTOT.KD" THEN "Investment"
    --WHEN indicator_code = "NE.GDI.FTOT.KN" THEN "Investment_alt"
END AS Indicator,
year Year,
value, 
Region, 
`Income Group`
FROM bigquery-public-data.world_bank_wdi.indicators_data data 
LEFT JOIN country_data
ON data.country_code = country_data.country_code
WHERE indicator_code IN (
    "NE.CON.TOTL.CD", --Consumption
    "NE.CON.TOTL.CN" --Consumption-alt 
    --"NE.GDI.FTOT.KD", --Investment
    --"NE.GDI.FTOT.KN" --Investment-alt 
  ) 
AND country_data.Region IS NOT NULL
AND country_data.`Income Group` IS NOT NULL
ORDER BY country_code, indicator, year
"""

In [45]:
query_job = client.query(QUERY)  # API request
df = query_job.to_dataframe()

In [46]:
df['value'].isna().sum()

0

In [47]:
df.head()

,country_code,Name,Indicator,Year,value,Region,Income Group
0,ABW,Aruba,Consumption,1995,9.301676e+08,Latin America & Caribbean,High income
1,ABW,Aruba,Consumption,1996,1.015642e+09,Latin America & Caribbean,High income
2,ABW,Aruba,Consumption,1997,1.125140e+09,Latin America & Caribbean,High income
3,ABW,Aruba,Consumption,1998,1.203911e+09,Latin America & Caribbean,High income
4,ABW,Aruba,Consumption,1999,1.271508e+09,Latin America & Caribbean,High income


In [48]:
df2 = df.copy()

In [49]:
import pandas as pd
mux = pd.MultiIndex.from_product([df2['country_code'].unique(), df2['Year'].unique(), df2['Indicator'].unique()],
                                  names=('country_code','Year', 'Indicator')).sort_values()
mux


MultiIndex([('ABW', 1960,     'Consumption'),
            ('ABW', 1960, 'Consumption_alt'),
            ('ABW', 1961,     'Consumption'),
            ('ABW', 1961, 'Consumption_alt'),
            ('ABW', 1962,     'Consumption'),
            ('ABW', 1962, 'Consumption_alt'),
            ('ABW', 1963,     'Consumption'),
            ('ABW', 1963, 'Consumption_alt'),
            ('ABW', 1964,     'Consumption'),
            ('ABW', 1964, 'Consumption_alt'),
            ...
            ('ZWE', 2016,     'Consumption'),
            ('ZWE', 2016, 'Consumption_alt'),
            ('ZWE', 2017,     'Consumption'),
            ('ZWE', 2017, 'Consumption_alt'),
            ('ZWE', 2018,     'Consumption'),
            ('ZWE', 2018, 'Consumption_alt'),
            ('ZWE', 2019,     'Consumption'),
            ('ZWE', 2019, 'Consumption_alt'),
            ('ZWE', 2020,     'Consumption'),
            ('ZWE', 2020, 'Consumption_alt')],
           names=['country_code', 'Year', 'Indicator'], length=

In [50]:
import numpy as np

In [51]:
df2 = df2.set_index(['country_code','Year', 'Indicator']).reindex(mux, fill_value=np.nan).reset_index()

In [52]:
df2

,country_code,Year,Indicator,Name,value,Region,Income Group
0,ABW,1960,Consumption,NaN,NaN,NaN,NaN
1,ABW,1960,Consumption_alt,NaN,NaN,NaN,NaN
2,ABW,1961,Consumption,NaN,NaN,NaN,NaN
3,ABW,1961,Consumption_alt,NaN,NaN,NaN,NaN
4,ABW,1962,Consumption,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
23175,ZWE,2018,Consumption_alt,Zimbabwe,3.827253e+10,Sub-Saharan Africa,Lower middle income
23176,ZWE,2019,Consumption,Zimbabwe,1.800596e+10,Sub-Saharan Africa,Lower middle income
23177,ZWE,2019,Consumption_alt,Zimbabwe,1.749891e+11,Sub-Saharan Africa,Lower middle income
23178,ZWE,2020,Consumption,Zimbabwe,1.699446e+10,Sub-Saharan Africa,Lower middle income


In [53]:
# finding: investment alt is better
# do have less missing values when use inflation
# but no guarantee
df2['isna'] = df2['value'].isna()
df2.groupby(['Indicator'])['isna'].sum()

Indicator
Consumption        3585
Consumption_alt    3532
Name: isna, dtype: int64